In [ ]:
# Libraries
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pip install \
--trusted-host 192.168.210.244 \
--index-url http://192.168.210.244:8081/repository/pypi/simple/ \
snowflake-connector-python

In [2]:
import json, snowflake.connector

# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/be00e2c6-806c-45f8-84c3-3fc99f64b8d3/saml2?SAMLRequest=pZNLc9sgFEb%2FioauJaGHHYmxnHHtydRt2npiua8dRlc2NQIFkOX01xfL8Uy6SDbdMXAuHO4Hk9tTI7wjaMOVLFAUYOSBZKriclegTXnnZ8gzlsqKCiWhQE9g0O10YmgjWjLr7F4%2BwGMHxnpuI2nIsFCgTkuiqOGGSNqAIZaR9ezzPYkDTFqtrGJKoBclb1dQY0BbZ3gtqQx3entrWxKGfd8HfRIovQtjjHGI89BRZ%2BTdlT%2B5O73CRyFOz7wjHL56dnvP5aUFb2ltL5AhH8py5a%2B%2Brkvkza6qcyVN14Begz5yBpuH%2B4uAcQatTvMcx4FU2u596LRqIWg1P1ILgstDYKTqa0EPwFTTdtYdErhRWEMVCrXjrg%2FLRYHaA69GH%2Fs%2FzUY%2B%2FsTL8vdsCw2mc%2FaJJj9O28P3m3TB5ViVsGhHsGPI%2B3YNOj4HvTSmg6U8x2vdFI4TH0d%2BfFNGKcEJweMgT7NfyFu4eLmkdqi83mHwCBrOtD

('7.3.0', 'ICHT_RG_COVOAM_22016_PROD', 'ICHT_WH_COVOAM_22016_PROD')


True

## Problem

In [5]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.PROBLEM
'''
cur = ctx.cursor().execute(query)
problem_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [55]:
# Filter for disorders
filtered_problem_df = problem_df[['PROBLEM', 'PROBLEM_DESC']]
filtered_problem_df = filtered_problem_df[filtered_problem_df['PROBLEM_DESC'].str.contains("\(disorder\)")==True]
filtered_problem_df.drop_duplicates(inplace=True)
filtered_problem_df.reset_index(inplace=True, drop=True)

In [56]:
# Get count
counts = problem_df[['PROBLEM']].value_counts()
counts = counts.reset_index()
counts.columns = ['PROBLEM', 'count']
# Merge
filtered_problem_df2 = pd.merge(filtered_problem_df, counts, how='left', on='PROBLEM')

In [58]:
# Drop duplicate
filtered_problem_df2.drop_duplicates(subset=['PROBLEM'], inplace=True)
# Sort 
filtered_problem_df2.sort_values(by=['count'], ascending=False, inplace=True)
filtered_problem_df2.reset_index(inplace=True, drop=True)

In [66]:
# Drop none
filtered_problem_df3 = filtered_problem_df2[filtered_problem_df2['PROBLEM'].str.contains("ICD")==False]

In [89]:
# Filter for move than 5
filtered_problem_df4 = filtered_problem_df3[filtered_problem_df3['count'] >= 5]

In [79]:
# Save
filtered_problem_df4.to_csv('snomed_problem_codes_morethan5.csv', index=False)

## Riskfactor

In [80]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.RISKFACTORS
'''
cur = ctx.cursor().execute(query)
riskfactor_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [83]:
# Filter for disorders
filtered_riskfactor_df = riskfactor_df[['SNOMED_CODE', 'CODE_DESCRIPTION']]
filtered_riskfactor_df = filtered_riskfactor_df[filtered_riskfactor_df['CODE_DESCRIPTION'].str.contains("\(disorder\)")==True]
filtered_riskfactor_df.drop_duplicates(inplace=True)
filtered_riskfactor_df.reset_index(inplace=True, drop=True)

In [84]:
# Get count
counts = riskfactor_df[['SNOMED_CODE']].value_counts()
counts = counts.reset_index()
counts.columns = ['SNOMED_CODE', 'count']
# Merge
filtered_riskfactor_df2 = pd.merge(filtered_riskfactor_df, counts, how='left', on='SNOMED_CODE')

In [85]:
# Drop duplicates
filtered_riskfactor_df2.drop_duplicates(subset=['SNOMED_CODE'], inplace=True)
# Sort 
filtered_riskfactor_df2.sort_values(by=['count'], ascending=False, inplace=True)
filtered_riskfactor_df2.reset_index(inplace=True, drop=True)

In [87]:
# Drop none
filtered_riskfactor_df3 = filtered_riskfactor_df2[filtered_riskfactor_df2['SNOMED_CODE'].str.contains("ICD")==False]

In [92]:
# Filter for move than 5
filtered_riskfactor_df4 = filtered_riskfactor_df3[filtered_riskfactor_df3['count'] >= 5]

In [96]:
# Save
filtered_riskfactor_df4.to_csv('snomed_riskfactor_codes_morethan5.csv', index=False)

## Diagnosis

In [3]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.EPISODES_DIAGNOSIS
'''
cur = ctx.cursor().execute(query)
diagnosis_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [113]:
# Filter for disorders
filtered_diagnosis_df = diagnosis_df[['DIAGNOSIS_CODE_SNOMED', 'DIAGNOSIS_DESCRIPTION_SNOMED']]
filtered_diagnosis_df = filtered_diagnosis_df[filtered_diagnosis_df['DIAGNOSIS_DESCRIPTION_SNOMED'].str.contains("\(disorder\)")==True]
filtered_diagnosis_df.drop_duplicates(inplace=True)
filtered_diagnosis_df.reset_index(inplace=True, drop=True)
# Get count
counts = diagnosis_df[['DIAGNOSIS_CODE_SNOMED']].value_counts()
counts = counts.reset_index()
counts.columns = ['DIAGNOSIS_CODE_SNOMED', 'count']
# Merge
filtered_diagnosis_df2 = pd.merge(filtered_diagnosis_df, counts, how='left', on='DIAGNOSIS_CODE_SNOMED')
# Drop duplicates
filtered_diagnosis_df2.drop_duplicates(subset=['DIAGNOSIS_CODE_SNOMED'], inplace=True)
# Sort 
filtered_diagnosis_df2.sort_values(by=['count'], ascending=False, inplace=True)
filtered_diagnosis_df2.reset_index(inplace=True, drop=True)
# Drop none
filtered_diagnosis_df3 = filtered_diagnosis_df2[filtered_diagnosis_df2['DIAGNOSIS_CODE_SNOMED'].str.contains("ICD")==False]
# Filter for move than 5
filtered_diagnosis_df4 = filtered_diagnosis_df3[filtered_diagnosis_df3['count'] >= 5]

In [ ]:
# Remove SNOMED string
filtered_diagnosis_df4['DIAGNOSIS_CODE_SNOMED'] = filtered_diagnosis_df4['DIAGNOSIS_CODE_SNOMED'].str.replace('SNOMED!', '')

In [117]:
# Save
filtered_diagnosis_df4.to_csv('snomed_diagnosis_codes_morethan5.csv', index=False)